# LLMalMorph Test trên Kaggle

Notebook này test hệ thống LLMalMorph với C.rar và CPP.rar trên Kaggle environment.

## 📋 Hướng Dẫn

1. **Set API Key**: Thay `your-mistral-api-key-here` bằng API key thật của bạn ở cell 2
2. **Run All Cells**: Chạy tất cả cells - notebook sẽ tự động:
   - Clone repository từ GitHub (có cả code và dataset)
   - Extract RAR files từ repository
   - Test hệ thống

**Repository**: https://github.com/quanturong/LLMalMorph2


## 1. Install Dependencies


In [ ]:
# Install dependencies
!pip install -q mistralai requests tree-sitter tree-sitter-c tree-sitter-cpp rarfile ollama

print("✓ Dependencies installed")


## 2. Setup Environment & API Key


In [ ]:
# Setup environment
import os
import sys
from pathlib import Path

# ⚠️ QUAN TRỌNG: Thay "your-mistral-api-key-here" bằng API key thật của bạn
MISTRAL_API_KEY = "your-mistral-api-key-here"  # ← THAY ĐỔI Ở ĐÂY

if MISTRAL_API_KEY != "your-mistral-api-key-here":
    os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY
    print("✓ API Key set")
else:
    print("⚠️  WARNING: API Key chưa được set!")
    print("   Vui lòng thay 'your-mistral-api-key-here' bằng API key thật")

# Repository sẽ được clone ở cell tiếp theo
REPO_DIR = "/kaggle/working/LLMalMorph2"
print(f"✓ Repository sẽ được clone vào: {REPO_DIR}")


## 3. Clone Repository


In [ ]:
# Clone repository (có cả code và dataset C.rar, CPP.rar)
REPO_URL = "https://github.com/quanturong/LLMalMorph2.git"
REPO_DIR = "/kaggle/working/LLMalMorph2"

if not os.path.exists(REPO_DIR):
    print("Cloning repository (có cả code và dataset)...")
    !git clone {REPO_URL} {REPO_DIR}
    print("✓ Repository cloned")
else:
    print("✓ Repository already exists")

# Add src to path
sys.path.insert(0, f'{REPO_DIR}/src')
print(f"✓ Added {REPO_DIR}/src to Python path")


## 4. Build Tree-sitter


In [ ]:
# Build tree-sitter từ source (nếu cần)
from tree_sitter import Language

# Try to use pre-built libraries first
try:
    import tree_sitter_c as ts_c
    import tree_sitter_cpp as ts_cpp
    C_LANGUAGE = Language(ts_c.language())
    CPP_LANGUAGE = Language(ts_cpp.language())
    print("✓ Using pre-built tree-sitter libraries")
except ImportError:
    print("Pre-built libraries not available, building from source...")
    
    # Clone tree-sitter-c nếu chưa có
    if not os.path.exists('tree-sitter-c'):
        !git clone --branch v0.20.2 https://github.com/tree-sitter/tree-sitter-c.git
        print("✓ Cloned tree-sitter-c")
    
    # Build library
    os.makedirs('build', exist_ok=True)
    
    if not os.path.exists('build/my-languages.so'):
        try:
            Language.build_library(
                'build/my-languages.so',
                ['tree-sitter-c']
            )
            print("✓ Built tree-sitter library")
        except Exception as e:
            print(f"⚠️  Build failed: {e}")
            print("   Will try to use pre-built libraries")
    else:
        print("✓ Tree-sitter library already exists")


## 5. Extract RAR Files


In [ ]:
# Extract RAR files từ repository đã clone
import rarfile
import zipfile
import glob

REPO_DIR = "/kaggle/working/LLMalMorph2"

# RAR files nằm trong repo
c_rar_path = f"{REPO_DIR}/C.rar"
cpp_rar_path = f"{REPO_DIR}/CPP.rar"

def extract_zip_files(directory):
    """Extract tất cả .zip files trong directory và subdirectories"""
    zip_files = glob.glob(f"{directory}/**/*.zip", recursive=True)
    print(f"  Found {len(zip_files)} .zip files to extract")
    
    for zip_path in zip_files:
        try:
            extract_dir = zip_path.rsplit('.', 1)[0]  # Remove .zip extension
            os.makedirs(extract_dir, exist_ok=True)
            
            with zipfile.ZipFile(zip_path, 'r') as zf:
                zf.extractall(extract_dir)
            print(f"    ✓ Extracted {os.path.basename(zip_path)}")
        except Exception as e:
            print(f"    ✗ Failed to extract {zip_path}: {e}")

# Extract C.rar
if os.path.exists(c_rar_path):
    extract_c_dir = f"{REPO_DIR}/extracted_C"
    os.makedirs(extract_c_dir, exist_ok=True)
    
    try:
        with rarfile.RarFile(c_rar_path) as rf:
            # List files trong RAR để debug
            file_list = rf.namelist()
            print(f"✓ C.rar contains {len(file_list)} files/folders")
            if file_list:
                print(f"  First few items: {file_list[:5]}")
            
            rf.extractall(extract_c_dir)
        print(f"✓ Extracted C.rar from repository")
        print(f"  → {extract_c_dir}")
        
        # Extract các .zip files bên trong
        print("\nExtracting .zip files inside C.rar...")
        extract_zip_files(extract_c_dir)
        
        # List extracted files để verify
        extracted_files = glob.glob(f"{extract_c_dir}/**/*.c", recursive=True)
        print(f"\n  Found {len(extracted_files)} .c files after extraction")
        if extracted_files:
            print(f"  Sample: {extracted_files[0]}")
        
    except Exception as e:
        print(f"✗ Failed to extract C.rar: {e}")
        import traceback
        traceback.print_exc()
else:
    print(f"⚠️  C.rar not found at {c_rar_path}")
    print("   Repository có thể chưa được clone đúng cách")

# Extract CPP.rar
if os.path.exists(cpp_rar_path):
    extract_cpp_dir = f"{REPO_DIR}/extracted_CPP"
    os.makedirs(extract_cpp_dir, exist_ok=True)
    
    try:
        with rarfile.RarFile(cpp_rar_path) as rf:
            # List files trong RAR để debug
            file_list = rf.namelist()
            print(f"\n✓ CPP.rar contains {len(file_list)} files/folders")
            if file_list:
                print(f"  First few items: {file_list[:5]}")
            
            rf.extractall(extract_cpp_dir)
        print(f"✓ Extracted CPP.rar from repository")
        print(f"  → {extract_cpp_dir}")
        
        # Extract các .zip files bên trong
        print("\nExtracting .zip files inside CPP.rar...")
        extract_zip_files(extract_cpp_dir)
        
        # List extracted files để verify
        extracted_files = glob.glob(f"{extract_cpp_dir}/**/*.cpp", recursive=True)
        extracted_files += glob.glob(f"{extract_cpp_dir}/**/*.cxx", recursive=True)
        extracted_files += glob.glob(f"{extract_cpp_dir}/**/*.cc", recursive=True)
        print(f"\n  Found {len(extracted_files)} .cpp/.cxx/.cc files after extraction")
        if extracted_files:
            print(f"  Sample: {extracted_files[0]}")
        
    except Exception as e:
        print(f"✗ Failed to extract CPP.rar: {e}")
        import traceback
        traceback.print_exc()
else:
    print(f"⚠️  CPP.rar not found at {cpp_rar_path}")
    print("   Repository có thể chưa được clone đúng cách")


In [ ]:
# Tìm tất cả C và CPP files từ extracted folders
REPO_DIR = "/kaggle/working/LLMalMorph2"

def find_source_files(directory, extensions):
    """Tìm source files trong directory"""
    directory = Path(directory)
    if not directory.exists():
        return []
    
    files = []
    for ext in extensions:
        files.extend(directory.rglob(f'*{ext}'))
    
    return sorted(files)

# Tìm C files
c_files = []
extract_c_dir = f"{REPO_DIR}/extracted_C"
if os.path.exists(extract_c_dir):
    c_files = find_source_files(extract_c_dir, ['.c'])
    print(f"✓ Found {len(c_files)} C files in {extract_c_dir}")
    if c_files:
        print(f"  First file: {c_files[0].name}")
else:
    print(f"⚠️  Extracted C directory not found: {extract_c_dir}")

# Tìm CPP files
cpp_files = []
extract_cpp_dir = f"{REPO_DIR}/extracted_CPP"
if os.path.exists(extract_cpp_dir):
    cpp_files = find_source_files(extract_cpp_dir, ['.cpp', '.cxx', '.cc'])
    print(f"✓ Found {len(cpp_files)} CPP files in {extract_cpp_dir}")
    if cpp_files:
        print(f"  First file: {cpp_files[0].name}")
else:
    print(f"⚠️  Extracted CPP directory not found: {extract_cpp_dir}")


## 7. Import Hệ Thống


In [ ]:
# Import hệ thống từ repository
REPO_DIR = "/kaggle/working/LLMalMorph2"

# Đảm bảo src trong path
if REPO_DIR not in sys.path:
    sys.path.insert(0, f'{REPO_DIR}/src')

try:
    from automation import IntegratedPipeline
    from config import get_config, setup_logging
    import logging
    
    # Setup logging
    logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
    config = get_config()
    setup_logging(config)
    
    print("✓ System imported successfully")
except ImportError as e:
    print(f"✗ Failed to import system: {e}")
    print(f"   Check if {REPO_DIR}/src exists")
    import traceback
    traceback.print_exc()


## 8. Test C File


In [ ]:
# Test với một C file
if c_files:
    test_file = c_files[0]
    print(f"\n{'='*60}")
    print(f"Testing: {test_file.name}")
    print(f"{'='*60}")
    
    # Read file
    with open(test_file, 'r', encoding='utf-8', errors='ignore') as f:
        source_code = f.read()
    
    # Initialize pipeline
    try:
        pipeline = IntegratedPipeline(
            language='c',
            llm_model='codestral-2508',
            api_key=os.environ.get('MISTRAL_API_KEY'),
            max_fix_attempts=1,
        )
        
        # Process
        results = pipeline.process_variant(
            source_file=str(test_file),
            variant_code=source_code,
            original_code=source_code,
            auto_fix=False,  # Skip auto-fix for quick test
            run_tests=False,  # Skip compilation test (no compiler on Kaggle)
        )
        
        # Print results
        comp = results.get('compilation', {})
        quality = results.get('quality', {})
        
        print(f"\nResults:")
        print(f"  Quality Score: {quality.get('quality_score', 0):.2f}")
        print(f"  Syntax Valid: {'✓' if quality.get('syntax_valid') else '✗'}")
        print(f"  Security Issues: {len(quality.get('security_issues', []))}")
        print(f"  Overall: {'✓ SUCCESS' if results.get('success') else '✗ FAILED'}")
        
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️  No C files found")


## 9. Test CPP File


In [ ]:
# Test với một CPP file
if cpp_files:
    test_file = cpp_files[0]
    print(f"\n{'='*60}")
    print(f"Testing: {test_file.name}")
    print(f"{'='*60}")
    
    # Read file
    with open(test_file, 'r', encoding='utf-8', errors='ignore') as f:
        source_code = f.read()
    
    # Initialize pipeline
    try:
        pipeline = IntegratedPipeline(
            language='cpp',
            llm_model='codestral-2508',
            api_key=os.environ.get('MISTRAL_API_KEY'),
            max_fix_attempts=1,
        )
        
        # Process
        results = pipeline.process_variant(
            source_file=str(test_file),
            variant_code=source_code,
            original_code=source_code,
            auto_fix=False,
            run_tests=False,
        )
        
        # Print results
        comp = results.get('compilation', {})
        quality = results.get('quality', {})
        
        print(f"\nResults:")
        print(f"  Quality Score: {quality.get('quality_score', 0):.2f}")
        print(f"  Syntax Valid: {'✓' if quality.get('syntax_valid') else '✗'}")
        print(f"  Security Issues: {len(quality.get('security_issues', []))}")
        print(f"  Overall: {'✓ SUCCESS' if results.get('success') else '✗ FAILED'}")
        
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️  No CPP files found")


In [ ]:
REPO_DIR = "/kaggle/working/LLMalMorph2"

print("\n" + "="*60)
print("TEST SUMMARY")
print("="*60)
print(f"\nRepository: {REPO_DIR}")
print(f"  Status: {'✓ Cloned' if os.path.exists(REPO_DIR) else '✗ Not found'}")
print(f"\nC Files Found: {len(c_files)}")
print(f"CPP Files Found: {len(cpp_files)}")
print(f"\nAPI Key Set: {'✓' if os.environ.get('MISTRAL_API_KEY') and os.environ.get('MISTRAL_API_KEY') != 'your-mistral-api-key-here' else '✗'}")
print(f"Tree-sitter: {'✓' if os.path.exists('build/my-languages.so') or 'C_LANGUAGE' in globals() else '✗'}")
print(f"RAR Files: {'✓' if os.path.exists(f'{REPO_DIR}/C.rar') and os.path.exists(f'{REPO_DIR}/CPP.rar') else '✗'}")
print("\n" + "="*60)
